In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import xgboost as xgb
import lightgbm as lgb


from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
)

In [2]:
def build_advanced_nn(input_shape):
    model = models.Sequential()
    model.add(layers.Dense(128, activation="relu", input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Dense(32, activation="relu"))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.4))

    model.add(layers.Dense(1, activation="sigmoid"))

    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [3]:
train_metadata_path = (
    "E:/W/ML NUST/(T5) CNN, RNN, LSTM/isic-2024-data/train-metadata.csv"
)
test_metadata_path = "E:/W/ML NUST/(T5) CNN, RNN, LSTM/isic-2024-data/test-metadata.csv"

train_metadata = pd.read_csv(train_metadata_path)
test_metadata = pd.read_csv(test_metadata_path)

train_metadata.head()

C:\Users\jahme\AppData\Local\Temp\ipykernel_1140\2963458992.py:6: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  train_metadata = pd.read_csv(train_metadata_path)


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,lesion_id,iddx_full,iddx_1,iddx_2,iddx_3,iddx_4,iddx_5,mel_mitotic_index,mel_thick_mm,tbp_lv_dnn_lesion_confidence
0,ISIC_0015670,0,IP_1235828,60.0,male,lower extremity,3.04,TBP tile: close-up,3D: white,20.244422,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,97.517282
1,ISIC_0015845,0,IP_8170065,60.0,male,head/neck,1.10,TBP tile: close-up,3D: white,31.712570,...,IL_6727506,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,3.141455
2,ISIC_0015864,0,IP_6724798,60.0,male,posterior torso,3.40,TBP tile: close-up,3D: XP,22.575830,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.804040
3,ISIC_0015902,0,IP_4111386,65.0,male,anterior torso,3.22,TBP tile: close-up,3D: XP,14.242329,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,99.989998
4,ISIC_0024200,0,IP_8313778,55.0,male,anterior torso,2.73,TBP tile: close-up,3D: white,24.725520,...,NaN,Benign,Benign,NaN,NaN,NaN,NaN,NaN,NaN,70.442510


In [4]:
# Checking for cols. with missing values.
missing_values = train_metadata.isnull().sum()
print("Missing Values=\n", missing_values)

Missing Values=
 isic_id                              0
target                               0
patient_id                           0
age_approx                        2798
sex                              11517
anatom_site_general               5756
clin_size_long_diam_mm               0
image_type                           0
tbp_tile_type                        0
tbp_lv_A                             0
tbp_lv_Aext                          0
tbp_lv_B                             0
tbp_lv_Bext                          0
tbp_lv_C                             0
tbp_lv_Cext                          0
tbp_lv_H                             0
tbp_lv_Hext                          0
tbp_lv_L                             0
tbp_lv_Lext                          0
tbp_lv_areaMM2                       0
tbp_lv_area_perim_ratio              0
tbp_lv_color_std_mean                0
tbp_lv_deltaA                        0
tbp_lv_deltaB                        0
tbp_lv_deltaL                        0
tbp_lv_d

In [5]:
columns_to_remove = [
    "lesion_id",
    "iddx_2",
    "iddx_3",
    "iddx_4",
    "iddx_5",
    "mel_mitotic_index",
    "mel_thick_mm",
]
train_metadata_cleaned = train_metadata.drop(columns=columns_to_remove, errors="ignore")
train_metadata_cleaned_no_nulls = train_metadata_cleaned.dropna()
train_metadata_cleaned_no_nulls["target"].value_counts()

target
0    381533
1       381
Name: count, dtype: int64

In [6]:
majority = train_metadata_cleaned_no_nulls[
    train_metadata_cleaned_no_nulls["target"] == 0
]
minority = train_metadata_cleaned_no_nulls[
    train_metadata_cleaned_no_nulls["target"] == 1
]
majority_class_downsample = resample(
    majority, replace=False, n_samples=len(minority), random_state=42
)
train_metadata_balanced = pd.concat([majority_class_downsample, minority])


print("Shpareee after balancing classes: ", train_metadata_balanced.shape)
print(
    "Class Dist. after balancing: \n", train_metadata_balanced["target"].value_counts()
)

Shpareee after balancing classes:  (762, 48)
Class Dist. after balancing: 
 target
0    381
1    381
Name: count, dtype: int64


In [7]:
train_metadata_balanced["sex"] = train_metadata_balanced["sex"].map(
    {"male": 1, "female": 0}
)

anatom_site_mapping = {
    "posterior torso": 1,
    "lower extremity": 2,
    "anterior torso": 3,
    "upper extremity": 4,
    "head/neck": 5,
}

tbp_lv_location_mapping = {
    "Torso Front Top Half": 1,
    "Torso Back Top Third": 2,
    "Head & Neck": 3,
    "Torso Back Middle Third": 4,
    "Left Leg - Lower": 5,
    "Right Leg - Lower": 6,
    "Torso Front Bottom Half": 7,
    "Left Arm - Upper": 8,
    "Left Leg - Upper": 9,
    "Right Arm - Upper": 10,
    "Right Leg - Upper": 11,
    "Left Arm - Lower": 12,
    "Right Arm - Lower": 13,
    "Torso Back Bottom Third": 14,
    "Left Leg": 15,
    "Right Leg": 16,
    "Left Arm": 17,
    "Right Arm": 18,
}


tbp_lv_location_simple_mapping = {
    "Torso Back": 1,
    "Torso Front": 2,
    "Left Leg": 3,
    "Head & Neck": 4,
    "Right Leg": 5,
    "Left Arm": 6,
    "Right Arm": 7,
}


train_metadata_balanced["anatom_site_general"] = train_metadata_balanced[
    "anatom_site_general"
].apply(lambda x: anatom_site_mapping.get(x, 0))

train_metadata_balanced["tbp_lv_location"] = train_metadata_balanced[
    "tbp_lv_location"
].apply(lambda x: tbp_lv_location_mapping.get(x, 0))

train_metadata_balanced["tbp_lv_location_simple"] = train_metadata_balanced[
    "tbp_lv_location_simple"
].apply(lambda x: tbp_lv_location_simple_mapping.get(x, 0))

In [8]:
train_metadata_balanced.head()


,isic_id,target,patient_id,age_approx,sex,anatom_site_general,clin_size_long_diam_mm,image_type,tbp_tile_type,tbp_lv_A,...,tbp_lv_symm_2axis,tbp_lv_symm_2axis_angle,tbp_lv_x,tbp_lv_y,tbp_lv_z,attribution,copyright_license,iddx_full,iddx_1,tbp_lv_dnn_lesion_confidence
44730,ISIC_1176500,0,IP_4013104,50.0,0,1,3.35,TBP tile: close-up,3D: white,21.195278,...,0.726667,60,-103.079308,1195.573975,227.486694,Memorial Sloan Kettering Cancer Center,CC-BY,Benign,Benign,99.990058
362429,ISIC_9047349,0,IP_9057861,80.0,1,2,2.82,TBP tile: close-up,3D: white,17.323980,...,0.460432,110,-170.181000,764.250000,29.220830,Memorial Sloan Kettering Cancer Center,CC-BY,Benign,Benign,99.999270
17389,ISIC_0500366,0,IP_6894380,45.0,0,1,4.81,TBP tile: close-up,3D: white,19.021500,...,0.207865,15,66.219480,1457.846000,78.279170,Memorial Sloan Kettering Cancer Center,CC-BY,Benign,Benign,99.999990
367815,ISIC_9176171,0,IP_6422845,70.0,1,1,4.19,TBP tile: close-up,3D: white,19.501620,...,0.334615,55,-39.389570,1057.866000,169.172800,Memorial Sloan Kettering Cancer Center,CC-BY,Benign,Benign,99.997030
192982,ISIC_4857877,0,IP_7331742,65.0,1,2,2.52,TBP tile: close-up,3D: XP,21.714430,...,0.185185,55,-150.631100,749.153000,-3.548462,"Department of Dermatology, Hospital Clínic de ...",CC-BY-NC,Benign,Benign,99.999950


In [9]:
train_metadata_balanced["age_sex_interaction"] = (
    train_metadata_balanced["age_approx"] * train_metadata_balanced["sex"]
)

In [10]:
X = train_metadata_balanced.drop(
    columns=[
        "isic_id",
        "target",
        "patient_id",
        "image_type",
        "tbp_tile_type",
        "attribution",
        "copyright_license",
        "iddx_full",
        "iddx_1",
        "tbp_lv_dnn_lesion_confidence",
    ]
)
y = train_metadata_balanced["target"]

In [11]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

In [12]:
train_dataset = (
    tf.data.Dataset.from_tensor_slices((X_train, y_train))
    .batch(64)
    .shuffle(buffer_size=1024)
)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

In [13]:
print(X_train.shape)

(609, 39)


In [14]:
input_shape = (39,)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3
)
model = build_advanced_nn(input_shape)
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[early_stopping, reduce_lr],
)

c:\Users\jahme\anaconda3\envs\PracConda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 24s 90ms/step - accuracy: 0.5240 - loss: 0.8560 - val_accuracy: 0.7124 - val_loss: 0.6102 - learning_rate: 0.0010
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6381 - loss: 0.6899 - val_accuracy: 0.7908 - val_loss: 0.5738 - learning_rate: 0.0010
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6816 - loss: 0.6501 - val_accuracy: 0.7647 - val_loss: 0.5475 - learning_rate: 0.0010
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7121 - loss: 0.5710 - val_accuracy: 0.7451 - val_loss: 0.5298 - learning_rate: 0.0010
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7399 - loss: 0.5583 - val_accuracy: 0.7516 - val_loss: 0.5187 - learning_rate: 0.0010
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7617 - loss: 0.5175 - val_accuracy: 0.7516 - val_loss: 0.5106 - learning_rate: 0.0010
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.7545 - loss: 0.5160 - val_accura

In [15]:
val_loss, val_accuracy = model.evaluate(val_dataset)
print(f"Validation Accuracy: {val_accuracy:.4f}")

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.8281 - loss: 0.3797

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7945 - loss: 0.4363 
Validation Accuracy: 0.7843


In [16]:
rf_model = RandomForestClassifier(
    n_estimators=100, max_depth=7, min_samples_split=10, random_state=42
)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, min_samples_split=10, random_state=42)

In [17]:
# RandomForest evaluation
y_val_pred_rf = rf_model.predict_proba(X_val)[:, 1]
roc_auc_rf = roc_auc_score(y_val, y_val_pred_rf)
print(f"RandomForest ROC AUC Score: {roc_auc_rf:.4f}")

RandomForest ROC AUC Score: 0.9186


In [18]:
test_metadata_cleaned = test_metadata.drop(columns=columns_to_remove, errors="ignore")
test_metadata_cleaned["sex"] = test_metadata_cleaned["sex"].map(
    {"male": 1, "female": 0}
)
test_metadata_cleaned["anatom_site_general"] = test_metadata_cleaned[
    "anatom_site_general"
].apply(lambda x: anatom_site_mapping.get(x, 0))
test_metadata_cleaned["tbp_lv_location"] = test_metadata_cleaned[
    "tbp_lv_location"
].apply(lambda x: tbp_lv_location_mapping.get(x, 0))
test_metadata_cleaned["tbp_lv_location_simple"] = test_metadata_cleaned[
    "tbp_lv_location_simple"
].apply(lambda x: tbp_lv_location_simple_mapping.get(x, 0))

test_metadata_cleaned["age_sex_interaction"] = (
    test_metadata_cleaned["age_approx"] * test_metadata_cleaned["sex"]
)

In [19]:
X_test = test_metadata_cleaned[X.columns]  # Align columns with training set
X_test_scaled = scaler.transform(X_test)

In [20]:
test_predictions = rf_model.predict_proba(X_test_scaled)[:, 1]
test_predicted_labels = (test_predictions > 0.5).astype(int)

In [21]:
submission = pd.DataFrame(
    {"isic_id": test_metadata["isic_id"], "target": test_predicted_labels}
)
submission.to_csv(
    "C:/Users/jahme/Desktop/Prac/Kaggle/ISIC_Skin_Cancer/output/submission.csv",
    index=False,
)